In [7]:
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import time 
import datetime
from datetime import timezone

In [2]:
pw = pd.read_parquet('part-00000-49a36603-9035-47f4-b73e-eae8d28aa10a-c000.gz.parquet')
pw.head()
outage = pw[pw['is_powered'] == True]
outage = outage[['time', 'location_latitude', 'location_longitude']]
outage.head()

In [88]:
jul_1 = outage[outage['time'] < datetime.datetime(2018, 7, 1, 0, 1)]
jul_1['time'] = jul_1['time'].apply(lambda x: x.replace(tzinfo=timezone.utc).timestamp())
len(jul_1)

In [89]:
# expanded = pw.join(pd.DataFrame(pw_time['time'].values.tolist(), columns=['year','month', 'day', 'hour', 'min', 'sec', 'sec1', 'sec2', 'sec3'], index=pw_time.index))
# expanded = expanded[['powered_longitude', 'powered_latitude', 'outage_longitude', 'outage_latitude', 'year','month', 'day', 'hour', 'min', 'sec']]
# expanded

In [100]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
from sklearn import metrics 

pw_cluster = StandardScaler().fit_transform(jul_1)
db = DBSCAN(eps=.9, algorithm='ball_tree').fit(pw_cluster)
labels = db.labels_

In [113]:
n_noise_ = list(labels).count(-1)
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_, n_clusters_ 

In [114]:
labels

In [115]:
print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)
# print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
# print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
# print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
# print("Adjusted Rand Index: %0.3f"
#       % metrics.adjusted_rand_score(labels_true, labels))
# print("Adjusted Mutual Information: %0.3f"
#       % metrics.adjusted_mutual_info_score(labels_true, labels,
#                                            average_method='arithmetic'))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(pw_cluster, labels))

In [116]:
#check with raw data of outages and times that they went out and then cluster based on outages. then see which ones seem to be the most reliably clustering 
jul_1['labels'] = labels
jul_1.head()

In [117]:
#pw_time[['outage_latitude', 'outage_longitude', 'labels']].groupby(['outage_latitude', 'outage_longitude']).first()

In [118]:
jul_1_0 = jul_1[jul_1['labels'] == 0]
jul_1_1 = jul_1[jul_1['labels'] == 1]
jul_1_2 = jul_1[jul_1['labels'] == 2]
jul_1_3 = jul_1[jul_1['labels'] == 3]
jul_1_4 = jul_1[jul_1['labels'] == 4]
jul_1_unlabeled = jul_1[jul_1['labels'] == -1]

In [120]:
# plt.scatter(pw_time['powered_longitude'], pw_time['powered_latitude'], c='b', label='powered')
plt.figure(figsize=(10,10))
plt.scatter(jul_1_0['location_longitude'], jul_1_0['location_latitude'], c='r', label='cluster 1')
plt.scatter(jul_1_1['location_longitude'], jul_1_1['location_latitude'], c='b', label='cluster 2')
plt.scatter(jul_1_2['location_longitude'], jul_1_2['location_latitude'], c='g', label='cluster 3')
plt.scatter(jul_1_3['location_longitude'], jul_1_3['location_latitude'], c='k', label='cluster 4')
plt.scatter(jul_1_4['location_longitude'], jul_1_4['location_latitude'], c='m', label='cluster 5')
plt.scatter(jul_1_unlabeled['location_longitude'], jul_1_unlabeled['location_latitude'], c='y', label='noise')
plt.title('Clustered Outages on 7/1/18 00:01:00')
plt.legend()
plt.xlabel('longitude')
plt.ylabel('latitude')

In [ ]:
plt.scatter(pw_time['powered_longitude'], pw_time['powered_latitude'], c='b', label='powered')
plt.scatter(pw_time['outage_longitude'], pw_time['outage_latitude'], c='r', label='outage')
plt.title('Powered Sensors vs Outages')
plt.legend()
plt.xlabel('longitude')
plt.ylabel('latitude')